In [1]:
import os
import sys
import pandas as pd

sys.path.append("/users/ejlaird/Projects/SmartCADD/")

import smartcadd
from smartcadd.dataset import IterableDataset
from smartcadd.pipeline import BasicCompoundPipeline

# set working directory
os.chdir("/users/ejlaird/Projects/SmartCADD/examples/")

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (/lustre/work/client/users/ejlaird/.conda/envs/smartcadd-test/lib/python3.11/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/lustre/work/client/users/ejlaird/.conda/envs/smartcadd-test/lib/python3.11/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax mo

## Building a Pipeline out of Dummy filters

### Create a Dataset Iterator

In [2]:
dataset = IterableDataset(
    root_dir="/work/users/ejlaird/data/chemistry",
    batch_size=10,
)
len(dataset)

50000

### Build the pipeline

In [3]:
from smartcadd.filters import DummyFilter

pipeline = BasicCompoundPipeline(
    data_loader=dataset,
    filters=[
        DummyFilter(),
        DummyFilter(),
    ],
)

print(pipeline)


BasicCompoundPipeline: [ (1) DummyFilter -> (2) DummyFilter -> Done ]


### Loop through dataset iterator and run filters for each batch

In [4]:
pipeline_results = pipeline.run_filters(steps=1)
pipeline_results

[{'smiles': 'CC(C)(O)C1CC[NH+](Cc2ccc3nc(-c4c5cc[nH]c5ccc4F)nc(N4CCOCC4)c3n2)CC1', 'id': 'ZINC000072316390', 'descriptors': {'status': 'OK', 'MolWt': 505.6180000000004, 'MolLogP': 2.714600000000001, 'NumHDonors': 3, 'NumHAcceptors': 6, 'TPSA': 91.60000000000001, 'CalcNumRotatableBonds': 5, 'NumAromaticRings': 4}, 'ring_system_descriptors': {'total_N_aro_members': 23, 'total_N_ali_members': 12, 'total_aro_N_count': 4, 'total_aro_O_count': 0, 'total_ali_O_count': 1, 'total_ali_N_count': 2}},
 {'smiles': 'Clc1nc(-c2ccccn2)c(-c2ccccn2)nc1N1CC[N@@H+]2C[C@H](OCc3cccnc3)C[C@H]2C1', 'id': 'ZINC001164868914', 'descriptors': {'status': 'OK', 'MolWt': 501.01400000000035, 'MolLogP': 2.7116000000000007, 'NumHDonors': 1, 'NumHAcceptors': 7, 'TPSA': 81.36, 'CalcNumRotatableBonds': 6, 'NumAromaticRings': 4}, 'ring_system_descriptors': {'total_N_aro_members': 24, 'total_N_ali_members': 11, 'total_aro_N_count': 5, 'total_aro_O_count': 0, 'total_ali_O_count': 0, 'total_ali_N_count': 3}},
 {'smiles': 'Clc

## Let's build a basic pipeline with real filters

### Wrap a pretrained model for use in ModelFilter

In [5]:
from smartcadd.model_wrappers import AttentiveFPWrapper

MODEL_PARAMS_PATH = (
    "/work/users/ejlaird/models/chemistry/attentive_fp_model_params.pt"
)

model_wrapper = AttentiveFPWrapper(
    model_params_path=MODEL_PARAMS_PATH,
    mode="classification",
    n_tasks=1,
    dropout=0.3,
    num_timesteps=2,  # defaut=2
    num_layers=2,  # default = 2
    graph_feat_size=200,  # default = 200
    number_atom_features=30,  # default = 30
    number_bond_features=11,  # default = 11
    batch_size=100,
    learning_rate=0.001,
    regularization_loss=None,
    device="cpu",
)
model_wrapper.load()

### Build basic pipeline and add ModelFilter

In [6]:
from smartcadd.filters import ModelFilter, ADMETFilter

deep_learning_pipeline = BasicCompoundPipeline(
    data_loader=dataset,
    filters=[
        ModelFilter(
            model_wrapper=model_wrapper,
            target=1,
            threshold=0.5,
            output_dir="./data",
            save_results=True,
        ),
    ],
)
print(deep_learning_pipeline)


BasicCompoundPipeline: [ (1) ModelFilter -> Done ]


### Run the pipeline


In [7]:
deep_learning_pipeline.run_filters(steps=1)

[{'smiles': 'O=C(Nc1cccc(C2=[NH+]CCN2)c1)c1ccc2cc(C(=O)Nc3cccc(C4=[NH+]CCN4)c3)ccc2c1', 'id': 'ZINC000001689092', 'descriptors': {'status': 'OK', 'MolWt': 504.5940000000002, 'MolLogP': 0.2052000000000021, 'NumHDonors': 6, 'NumHAcceptors': 4, 'TPSA': 110.2, 'CalcNumRotatableBonds': 6, 'NumAromaticRings': 4}, 'ring_system_descriptors': {'total_N_aro_members': 24, 'total_N_ali_members': 10, 'total_aro_N_count': 0, 'total_aro_O_count': 0, 'total_ali_O_count': 0, 'total_ali_N_count': 4}},
 {'smiles': 'O=C(Nc1ccc(C2=[NH+]CCN2)c(Cl)c1)c1ccc(C(=O)Nc2ccc(C3=[NH+]CCN3)c(Cl)c2)cc1', 'id': 'ZINC000001689095', 'descriptors': {'status': 'OK', 'MolWt': 523.4240000000002, 'MolLogP': 0.35880000000000206, 'NumHDonors': 6, 'NumHAcceptors': 4, 'TPSA': 110.2, 'CalcNumRotatableBonds': 6, 'NumAromaticRings': 3}, 'ring_system_descriptors': {'total_N_aro_members': 18, 'total_N_ali_members': 10, 'total_aro_N_count': 0, 'total_aro_O_count': 0, 'total_ali_O_count': 0, 'total_ali_N_count': 4}},
 {'smiles': 'O=C(Nc

### Check results in saved csv

In [8]:
deep_learning_results = pd.read_csv("./data/model_filtered.csv")
deep_learning_results[deep_learning_results["Prediction"] > 0.0]

,SMILES,ID,Prediction
5,O=C(Nc1cccc(C2=[NH+]CCN2)c1)c1ccc2cc(C(=O)Nc3c...,ZINC000001689092,0.998
6,O=C(Nc1ccc(C2=[NH+]CCN2)c(Cl)c1)c1ccc(C(=O)Nc2...,ZINC000001689095,0.837
7,O=C(Nc1ccc(C2=[NH+]CCN2)cc1)c1cc(Cl)c(C(=O)Nc2...,ZINC000001689504,0.833
15,O=C(Nc1cccc(C2=[NH+]CCN2)c1)c1ccc2cc(C(=O)Nc3c...,ZINC000001689092,0.998
16,O=C(Nc1ccc(C2=[NH+]CCN2)c(Cl)c1)c1ccc(C(=O)Nc2...,ZINC000001689095,0.837
...,...,...,...
917,O=C(/C=C/c1ccc(/C=C/C(=O)Nc2cccc(C3=[NH+]CCCN3...,ZINC000104051067,0.999
932,CCc1ccc(N2NC(=O)/C(=C/c3ccc(-c4ccc(S(=O)(=O)[N...,ZINC000002126864,0.001
945,O=C(Nc1cccc(C2=[NH+]CCN2)c1)c1ccc2cc(C(=O)Nc3c...,ZINC000001689092,0.998
946,O=C(Nc1ccc(C2=[NH+]CCN2)c(Cl)c1)c1ccc(C(=O)Nc2...,ZINC000001689095,0.837


## Building a custom filter using the Filter Interface

In [9]:
from smartcadd.filters import Filter
from random import random


class RandomFilter(Filter):
    """
    An example filter that randomly selects compounds
    """

    def __init__(self, threshold):
        super().__init__()

        self.threshold = threshold

    def run(self, batch):

        # select a random number of compounds to keep
        mask = [random() > self.threshold for _ in range(len(batch))]
        return [compound for compound, keep in zip(batch, mask) if keep]

### Append new filter to existing Pipeline

In [10]:
deep_learning_pipeline.append_filter(RandomFilter(threshold=0.3))
print(deep_learning_pipeline)


BasicCompoundPipeline: [ (1) ModelFilter -> (2) RandomFilter -> Done ]


In [11]:
filtered_compounds = deep_learning_pipeline.run_filters(steps=5)
print(f"Length of results after filtering: {len(filtered_compounds)}")

Length of results after filtering: 5


### ADMET Filtering

In [12]:
from smartcadd.filters import ADMETFilter

ALERT_COLLECTION_PATH = (
    "/work/users/ejlaird/data/chemistry/alert_collection.csv"
)

admet_pipeline = BasicCompoundPipeline(
    data_loader=dataset,
    filters=[
        ADMETFilter(
            alert_collection_path=ALERT_COLLECTION_PATH,
            output_dir="./data",
            save_results=True,
        ),
    ],
)
print(admet_pipeline)


BasicCompoundPipeline: [ (1) ADMETFilter -> Done ]


In [13]:
admet_filtered_compounds = admet_pipeline.run_filters(steps=5)
print(f"Length of batch after filtering: {len(admet_filtered_compounds)}")

Length of batch after filtering: 38


### Check results in saved csv

In [14]:
import pandas as pd

admet_results = pd.read_csv("./data/ADMET_filtered.csv")
admet_results[admet_results["keep"] == False]

,smiles,id,status,MolWt,MolLogP,NumHDonors,NumHAcceptors,TPSA,CalcNumRotatableBonds,NumAromaticRings,total_N_aro_members,total_N_ali_members,total_aro_N_count,total_aro_O_count,total_ali_O_count,total_ali_N_count,keep
38,COc1ccc(N2C(=O)/C(=C\c3cc(C)n(-c4ccc(Cl)c(C(=O...,ZINC000000681904,OK,538.989,2.95954,1,8,112.93,6,3,17,6,1,0,0,2,False
39,Cc1ccc(N2C(=O)/C(=C/c3cc(Br)c([O-])c(Br)c3)C(=...,ZINC000000714676,OK,509.199,3.73334,1,4,72.47,2,2,12,6,0,0,0,2,False
40,CCOc1ccc(N2C(=O)/C(=C/c3ccc(OCc4ccc(C(=O)[O-])...,ZINC000001201230,OK,531.566,2.86770,1,8,117.23,9,3,18,6,0,0,0,2,False
41,COc1cc(Br)c(Br)c([C@H]2Nc3sc4c(c3C(=O)N2)CC[N@...,ZINC000001250478,OK,504.224,2.41220,4,5,75.03,2,2,15,8,0,0,0,3,False
42,CCc1ccc(N2NC(=O)/C(=C/c3ccc(-c4ccc(S(=O)(=O)[N...,ZINC000002126864,OK,519.584,4.82100,1,7,123.68,7,4,22,5,1,1,0,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,CCOc1ccc(N2C(=O)/C(=C/c3ccc(OCc4cccc(C(=O)[O-]...,ZINC000002308579,OK,501.540,2.85910,1,7,108.00,8,3,18,6,0,0,0,2,False
666,COc1ccc(N2C(=O)/C(=C/c3cc(C)n(-c4ccc(Cl)c(C(=O...,ZINC000002783351,OK,508.963,2.95094,1,7,103.70,5,3,17,6,1,0,0,2,False
667,COc1cc(/C=C2\C(=O)NC(=S)N(c3ccc(Cl)cc3)C2=O)cc...,ZINC000002908281,OK,521.958,3.12240,1,7,108.00,7,3,18,6,0,0,0,2,False
668,CCOc1cc(C=C2C(=O)[N-]C(=S)NC2=O)cc(Br)c1OCc1cc...,ZINC000002933000,OK,505.326,4.03340,1,7,121.87,7,2,12,6,0,0,0,2,False


## 2D Pharmacophore Filtering

In [15]:
from smartcadd.data import Compound

template_compound_smiles = {
    "rilpivirine": "CC1=CC(=CC(=C1NC2=NC(=NC=C2)NC3=CC=C(C=C3)C#N)C)/C=C/C#N",
    "etravirine": "CC1=CC(=CC(=C1OC2=NC(=NC(=C2Br)N)NC3=CC=C(C=C3)C#N)C)C#N",
}

template_compounds = [
    Compound(smiles=smiles, id=drug)
    for drug, smiles in template_compound_smiles.items()
]

### Check minimum values of template compounds for 2D Pharmacophore Filtering

In [16]:
min_df = pd.concat(compound.to_df() for compound in template_compounds)
min_df.min(axis=0).to_dict()

{'smiles': 'CC1=CC(=CC(=C1NC2=NC(=NC=C2)NC3=CC=C(C=C3)C#N)C)/C=C/C#N',
 'id': 'etravirine',
 'status': 'OK',
 'MolWt': 366.4280000000001,
 'MolLogP': 4.717400000000003,
 'NumHDonors': 2,
 'NumHAcceptors': 6,
 'TPSA': 97.41999999999999,
 'CalcNumRotatableBonds': 4,
 'NumAromaticRings': 3,
 'total_N_aro_members': 18,
 'total_N_ali_members': 0,
 'total_aro_N_count': 2,
 'total_aro_O_count': 0,
 'total_ali_O_count': 0,
 'total_ali_N_count': 0}

In [17]:
from smartcadd.filters import PharmacophoreFilter2D

pharmacophore_pipeline = BasicCompoundPipeline(
    data_loader=dataset,
    filters=[
        PharmacophoreFilter2D(
            template_compounds=template_compounds,
            output_dir="./data",
            save_results=True,
        ),
    ],
)
print(pharmacophore_pipeline)


BasicCompoundPipeline: [ (1) PharmacophoreFilter2D -> Done ]


In [18]:
pharmacophore_filtered_compounds = pharmacophore_pipeline.run_filters(steps=20)
print(
    f"Length of batch after filtering: {len(pharmacophore_filtered_compounds)}"
)

Length of batch after filtering: 0


## Data Type Conversion Modules and Geometry Optimization

In [19]:
from smartcadd.modules import SMILETo3D, XTBOptimization

geometry_optimization_pipeline = BasicCompoundPipeline(
    data_loader=dataset,
    filters=[
        SMILETo3D(
            modify=True,
            output_dir="./data",
            save_results=True,
        ),
    ],
)
print(geometry_optimization_pipeline)
print("Saving 3D coordinates to ./data/3D_coordinates.csv")


BasicCompoundPipeline: [ (1) SMILETo3D -> Done ]
Saving 3D coordinates to ./data/3D_coordinates.csv


In [20]:
transformed_compounds = geometry_optimization_pipeline.run_filters(steps=1)

### Load newly created pdb file paths

In [21]:
pdb_csv = pd.read_csv("./data/3D_coordinates.csv")
pdb_csv

,SMILES,ID,PDB_PATH
0,CC(C)(O)C1CC[NH+](Cc2ccc3nc(-c4c5cc[nH]c5ccc4F...,ZINC000072316390,./data/3D_coordinates/ZINC000072316390.pdb
1,Clc1nc(-c2ccccn2)c(-c2ccccn2)nc1N1CC[N@@H+]2C[...,ZINC001164868914,./data/3D_coordinates/ZINC001164868914.pdb
2,Clc1nc(-c2ccccn2)c(-c2ccccn2)nc1N1CC[N@@H+]2C[...,ZINC001164868936,./data/3D_coordinates/ZINC001164868936.pdb
3,Clc1nc(-c2ccccn2)c(-c2ccccn2)nc1N1CC[N@@H+]2C[...,ZINC001164868939,./data/3D_coordinates/ZINC001164868939.pdb
4,Clc1nc(-c2ccccn2)c(-c2ccccn2)nc1N1CC[N@@H+]2C[...,ZINC001164868941,./data/3D_coordinates/ZINC001164868941.pdb
5,O=C(Nc1cccc(C2=[NH+]CCN2)c1)c1ccc2cc(C(=O)Nc3c...,ZINC000001689092,./data/3D_coordinates/ZINC000001689092.pdb
6,O=C(Nc1ccc(C2=[NH+]CCN2)c(Cl)c1)c1ccc(C(=O)Nc2...,ZINC000001689095,./data/3D_coordinates/ZINC000001689095.pdb
7,O=C(Nc1ccc(C2=[NH+]CCN2)cc1)c1cc(Cl)c(C(=O)Nc2...,ZINC000001689504,./data/3D_coordinates/ZINC000001689504.pdb
8,C[C@H]1C[NH+]=C(Cc2ccc(NC(=O)c3ccc(C(=O)Nc4ccc...,ZINC000001696457,./data/3D_coordinates/ZINC000001696457.pdb
9,C[C@@H]1C[NH+]=C(Cc2ccc(NC(=O)c3ccc(C(=O)Nc4cc...,ZINC000001696458,./data/3D_coordinates/ZINC000001696458.pdb


## Geometry Optimization using XTB

In [22]:
from smartcadd.modules import XTBOptimization

geometry_optimization_pipeline.append_filter(
    XTBOptimization(
        from_file=True,
        pdb_dir="./data/3D_coordinates",
        output_dir="./data",
        n_processes=16,
        save_results=True,
    )
)
print(geometry_optimization_pipeline)


BasicCompoundPipeline: [ (1) SMILETo3D -> (2) XTBOptimization -> Done ]


In [23]:
optimized_compounds = geometry_optimization_pipeline.run_filters(steps=1)

      -----------------------------------------------------------      
     |                   =====================                   |     
     |                           x T B                           |     
     |                   =====================                   |     
     |                         S. Grimme                         |     
     |          Mulliken Center for Theoretical Chemistry        |     
     |                    University of Bonn                     |     
      -----------------------------------------------------------      

   * xtb version 6.6.1 (8d0f1dd) compiled by 'conda@1efc2f54142f' on 2023-08-01

   xtb is free software: you can redistribute it and/or modify it under
   the terms of the GNU Lesser General Public License as published by
   the Free Software Foundation, either version 3 of the License, or
   (at your option) any later version.
   
   xtb is distributed in the hope that it will be useful,
   but WITHOUT ANY WARRANTY; w

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


 Shifting diagonal of input Hessian by    9.8568199799604263E-003
 Lowest  eigenvalues of input Hessian
    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000
    0.010000    0.010007    0.010185    0.010337    0.010598    0.010714
    0.011152    0.011233    0.011551    0.011905    0.013075    0.013558
 Highest eigenvalues
    2.035216    2.067337    2.076008    2.112333    2.120043    2.198847


........................................................................
.............................. CYCLE    1 ..............................
........................................................................

 iter      E             dE          RMSdq      gap      omega  full diag
   1   -101.5963995 -0.101596E+03  0.470E-04    1.51       0.0  T
   2   -101.5963995 -0.226093E-08  0.397E-04    1.51      31.8  T
   3   -101.5963995 -0.164877E-08  0.333E-04    1.51      37.8  T
     SCC iter.                  ...        0 min,  0.017 sec
     gradient               

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL



         #    Occupation            Energy/Eh            Energy/eV
      -------------------------------------------------------------
         1        2.0000           -1.1477478             -31.2318
       ...           ...                  ...                  ...
        85        2.0000           -0.4985147             -13.5653
        86        2.0000           -0.4882336             -13.2855
        87        2.0000           -0.4788603             -13.0305
        88        2.0000           -0.4582437             -12.4694
        89        2.0000           -0.4496702             -12.2361
        90        2.0000           -0.4441048             -12.0847
        91        2.0000           -0.4265475             -11.6069 (HOMO)
        92                         -0.3812343             -10.3739 (LUMO)
        93                         -0.3498751              -9.5206
        94                         -0.3445306              -9.3752
        95                         -0.3318615 

...

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL



          ...................................................
          :                      SETUP                      :
          :.................................................:
          :   optimization level            normal          :
          :   max. optcycles                   216          :
          :   ANC micro-cycles                  20          :
          :   degrees of freedom               210          :
          :.................................................:
          :   RF solver                   davidson          :
          :   write xtbopt.log                true          :
          :   linear?                        false          :
          :   energy convergence         0.5000000E-05 Eh   :
          :   grad. convergence          0.1000000E-02 Eh/α :
          :   maxmium RF displ.          1.0000000          :
          :   Hlow (freq-cutoff)         0.1000000E-01      :
          :   Hmax (freq-cutoff)         5.0000000          :
       

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


   1   -108.0020651 -0.108002E+03  0.102E-04    1.72       0.0  T
   2   -108.0020651 -0.933369E-10  0.627E-05    1.72     187.9  T
   3   -108.0020651 -0.111555E-10  0.402E-05    1.72     292.9  T
     SCC iter.                  ...        0 min,  0.021 sec
     gradient                   ...        0 min,  0.049 sec
 * total energy  :  -106.4077699 Eh     change       -0.1385501E-08 Eh
   gradient norm :     0.2547053 Eh/α   predicted     0.0000000E+00 (-100.00%)
   displ. norm   :     0.4316616 α      lambda       -0.6849588E-01
   maximum displ.:     0.1133898 α      in ANC's #50, #122, #123, ...

........................................................................
.............................. CYCLE    2 ..............................
........................................................................

 iter      E             dE          RMSdq      gap      omega  full diag
   1   -108.0863364 -0.108086E+03  0.316E-01    1.84       0.0  T
   2   -108.0887577 -0.242125E-

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
